In [227]:
import pandas as pd
df = pd.read_excel('data/PEPANNRES.xlsx')
df

,Geography,Population
0,"Abilene, TX Metro Area",165252
1,"Aguadilla-Isabela, PR Metro Area",339441
2,"Akron, OH Metro Area",703200
3,"Albany, GA Metro Area",153857
4,"Albany-Lebanon, OR Metro Area",116672
...,...,...
387,"Yauco, PR Metro Area",104954
388,"York-Hanover, PA Metro Area",434972
389,"Youngstown-Warren-Boardman, OH-PA Metro Area",565773
390,"Yuba City, CA Metro Area",166892


In [228]:
import pandas as pd
import sqlite3

# Define file paths
file_paths = {
    "B25003": "data/B25003.xlsx",     # Owner vs renter
    "B25002": "data/B25002.xlsx",     # Occupancy (vacancy rate)
    "B25077": "data/B25077.xlsx",     # Median Home Value
    "B19013": "data/B19013.xlsx",     # Median Household Income
    "B25070": "data/B25070.xlsx",     # Gross Rent as % Income
    "C40":     "data/C-40.xlsx",      # New housing units
    "PEPANNRES": "data/PEPANNRES.xlsx",  # Population
    "B25091": "data/B25091.xlsx",     # ✅ Owner Cost Burden (optional)
    "B25036": "data/B25036.xlsx",      # ✅ Year Structure Built (optional)
    "PopulationChange": "data/ChangeInPopulation.xlsx"

}

# Load and normalize Geography column in each file


dfs = {}
for key, path in file_paths.items():
    df = pd.read_excel(path, sheet_name=0)
    df.columns = df.columns.str.strip()  # Clean whitespace in column names

    if "Geography" in df.columns:
        df["Geography"] = df["Geography"].str.strip()
        df["Geography"] = df["Geography"].str.replace(" Metro Area", "", regex=False)
        df["Geography"] = df["Geography"].str.replace(" MSA", "", regex=False)
        df["Geography"] = df["Geography"].str.replace("–", "-", regex=False)
        df["Geography"] = df["Geography"].str.replace("—", "-", regex=False)
        df["Geography"] = df["Geography"].str.replace(u"\xa0", " ", regex=False)  # non-breaking space
        df["Geography"] = df["Geography"].str.replace("-", "-", regex=False)

        # ✅ Extract only the first place name (before comma or dash)
        df["Geography"] = df["Geography"].str.split(",|-").str[0].str.strip()

    dfs[key] = df




# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")

# Load each DataFrame into SQLite
for name, df in dfs.items():
    df.to_sql(name, conn, index=False, if_exists='replace')



# SQL query for joined dataset
sql_query = """
SELECT 
    B25003.Geography AS Metro_Area,
    B25003.[Total] AS Total_Housing_Units,
    B25003.[Owner occupied],
    B25003.[Renter occupied],
    B25002.[Vacant],
    B25077.[Median value (dollars)] AS Median_Home_Value,
    B19013.MedianHouseholdIncome,
    B25070.[50.0 percent or more] AS High_Rent_Burden,
    C40.[Total] AS New_Housing_Units,
    PEPANNRES.Population,
    PopulationChange.[Percent Change in Population (2020-2024)],
    B25091.owner_cost_burden,
    B25036.[Built 2010 or later] AS Recent_Units_Share
FROM B25003
LEFT JOIN B25002 ON B25003.Geography = B25002.Geography
LEFT JOIN B25077 ON B25003.Geography = B25077.Geography
LEFT JOIN B19013 ON B25003.Geography = B19013.Geography
LEFT JOIN B25070 ON B25003.Geography = B25070.Geography
LEFT JOIN C40 ON B25003.Geography = C40.Geography
LEFT JOIN PEPANNRES ON B25003.Geography = PEPANNRES.Geography
LEFT JOIN B25091 ON B25003.Geography = B25091.Geography
LEFT JOIN B25036 ON B25003.Geography = B25036.Geography
LEFT JOIN PopulationChange ON B25003.Geography = PopulationChange.Geography
GROUP BY PopulationChange.Geography
ORDER BY PEPANNRES.Population DESC
LIMIT 100



"""

# Execute and load the result into a DataFrame
housing_index_df = pd.read_sql_query(sql_query, conn)

# Preview or export
# housing_index_df.to_csv("Housing_Market_Index_Base.csv", index=False)

# Close SQLite connection
conn.close()

housing_index_df

,Metro_Area,Total_Housing_Units,Owner occupied,Renter occupied,Vacant,Median_Home_Value,MedianHouseholdIncome,High_Rent_Burden,New_Housing_Units,Population,Percent Change in Population (2020-2024),Owner_Cost_Burden,Recent_Units_Share
0,New York,"7,490,498","3,825,885","3,664,613","593,023",610200,95220,"983,573",56661.0,19052774,-0.007054,1220190,540955
1,Los Angeles,"4,542,998","2,189,902","2,353,096","285,894",867200,91960,"686,139",31151.0,12828837,-0.020709,747559,318295
2,Chicago,"3,671,659","2,420,840","1,250,819","240,243",316500,87071,"285,897",18511.0,9461105,-0.004315,597290,246398
3,Dallas,"2,959,385","1,769,613","1,189,772","212,902",373900,86860,"295,171",78705.0,6366542,0.092524,436350,724661
4,Philadelphia,"2,484,152","1,668,735","815,417","159,676",356700,86867,"212,866",36307.0,5965343,0.013670,386556,188296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Lexington,"222,167","135,320","86,847","11,382",289200,70027,"21,360",2943.0,478930,0.032033,22718,31403
96,Pensacola,"204,344","148,104","56,240","28,687",308600,78315,"12,679",4051.0,455139,0.056918,32809,39853
97,Visalia,"145,442","86,036","59,406","9,803",339800,73097,"15,209",1510.0,442179,0.022043,23492,16052
98,Fayetteville,"150,658","88,351","62,307","15,242",216900,61220,"14,154",2307.0,440121,0.018102,20811,23193


In [229]:
housing_index_df = housing_index_df.dropna()

housing_index_df

,Metro_Area,Total_Housing_Units,Owner occupied,Renter occupied,Vacant,Median_Home_Value,MedianHouseholdIncome,High_Rent_Burden,New_Housing_Units,Population,Percent Change in Population (2020-2024),Owner_Cost_Burden,Recent_Units_Share
0,New York,"7,490,498","3,825,885","3,664,613","593,023",610200,95220,"983,573",56661.0,19052774,-0.007054,1220190,540955
1,Los Angeles,"4,542,998","2,189,902","2,353,096","285,894",867200,91960,"686,139",31151.0,12828837,-0.020709,747559,318295
2,Chicago,"3,671,659","2,420,840","1,250,819","240,243",316500,87071,"285,897",18511.0,9461105,-0.004315,597290,246398
3,Dallas,"2,959,385","1,769,613","1,189,772","212,902",373900,86860,"295,171",78705.0,6366542,0.092524,436350,724661
4,Philadelphia,"2,484,152","1,668,735","815,417","159,676",356700,86867,"212,866",36307.0,5965343,0.013670,386556,188296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Lexington,"222,167","135,320","86,847","11,382",289200,70027,"21,360",2943.0,478930,0.032033,22718,31403
96,Pensacola,"204,344","148,104","56,240","28,687",308600,78315,"12,679",4051.0,455139,0.056918,32809,39853
97,Visalia,"145,442","86,036","59,406","9,803",339800,73097,"15,209",1510.0,442179,0.022043,23492,16052
98,Fayetteville,"150,658","88,351","62,307","15,242",216900,61220,"14,154",2307.0,440121,0.018102,20811,23193


### Feature Engineering
| **Feature**           | **Formula**                                                                       | **Description**                          |
| --------------------- | --------------------------------------------------------------------------------- | ---------------------------------------- |
| `Affordability_Ratio` | `Median_Home_Value / MedianHouseholdIncome`                                       | High = less affordable                   |
| `Vacancy_Rate`        | `Vacant / Total_Housing_Units`                                                    | Share of unoccupied housing              |
| `Rent_Burden_Rate`    | `High_Rent_Burden / Renter occupied` *(if `High_Rent_Burden` is not already a %)* | % renters burdened                       |
| `Owner_Burden_Rate`   | `Owner_Cost_Burden / Owner occupied` *(if not already a %)*                       | % homeowners burdened                    |
| `New_Units_Rate`      | `New_Housing_Units / Total_Housing_Units`                                         | Share of recently built housing stock    |
| `Pop_to_Unit_Ratio`   | `Population / Total_Housing_Units`                                                | High = higher pressure on housing supply |
| `Recent_Units_Share`  | Already calculated from B25036                                                    | % of housing built in 2010 or later      |

In [230]:
# Step 1: Define columns to convert
cols_to_convert = [
    'Median_Home_Value',
    'MedianHouseholdIncome',
    'Vacant',
    'Total_Housing_Units',
    'High_Rent_Burden',
    'Renter occupied',
    'Owner_Cost_Burden',
    'Owner occupied',
    'New_Housing_Units',
    'Population'
]

# Step 2: Remove commas and convert to numeric
for col in cols_to_convert:
    housing_index_df[col] = (
        housing_index_df[col]
        .astype(str)                          # Ensure string type
        .str.replace(',', '', regex=False)    # Remove commas
        .replace('', pd.NA)                   # Replace empty strings with NA
        .astype(float)                        # Convert to float
    )

# Step 3: Drop rows with any missing (NaN) values after conversion
housing_index_df = housing_index_df.dropna(subset=cols_to_convert)

# Step 4: Feature Engineering
housing_index_df['Affordability_Ratio'] = housing_index_df['Median_Home_Value'] / housing_index_df['MedianHouseholdIncome']
housing_index_df['Vacancy_Rate'] = housing_index_df['Vacant'] / housing_index_df['Total_Housing_Units']
housing_index_df['Rent_Burden_Rate'] = housing_index_df['High_Rent_Burden'] / housing_index_df['Renter occupied']
housing_index_df['Owner_Burden_Rate'] = housing_index_df['Owner_Cost_Burden'] / housing_index_df['Owner occupied']
housing_index_df['New_Units_Rate'] = housing_index_df['New_Housing_Units'] / housing_index_df['Total_Housing_Units']
housing_index_df['Pop_to_Unit_Ratio'] = housing_index_df['Population'] / housing_index_df['Total_Housing_Units']

housing_index_df

C:\Users\maktr\AppData\Local\Temp\ipykernel_24276\2887178175.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_index_df[col] = (
C:\Users\maktr\AppData\Local\Temp\ipykernel_24276\2887178175.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_index_df[col] = (
C:\Users\maktr\AppData\Local\Temp\ipykernel_24276\2887178175.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,Metro_Area,Total_Housing_Units,Owner occupied,Renter occupied,Vacant,Median_Home_Value,MedianHouseholdIncome,High_Rent_Burden,New_Housing_Units,Population,Percent Change in Population (2020-2024),Owner_Cost_Burden,Recent_Units_Share,Affordability_Ratio,Vacancy_Rate,Rent_Burden_Rate,Owner_Burden_Rate,New_Units_Rate,Pop_to_Unit_Ratio
0,New York,7490498.0,3825885.0,3664613.0,593023.0,610200.0,95220.0,983573.0,56661.0,19052774.0,-0.007054,1220190.0,540955,6.408318,0.079170,0.268398,0.318930,0.007564,2.543592
1,Los Angeles,4542998.0,2189902.0,2353096.0,285894.0,867200.0,91960.0,686139.0,31151.0,12828837.0,-0.020709,747559.0,318295,9.430187,0.062931,0.291590,0.341366,0.006857,2.823870
2,Chicago,3671659.0,2420840.0,1250819.0,240243.0,316500.0,87071.0,285897.0,18511.0,9461105.0,-0.004315,597290.0,246398,3.634965,0.065432,0.228568,0.246728,0.005042,2.576793
3,Dallas,2959385.0,1769613.0,1189772.0,212902.0,373900.0,86860.0,295171.0,78705.0,6366542.0,0.092524,436350.0,724661,4.304628,0.071941,0.248090,0.246579,0.026595,2.151306
4,Philadelphia,2484152.0,1668735.0,815417.0,159676.0,356700.0,86867.0,212866.0,36307.0,5965343.0,0.013670,386556.0,188296,4.106277,0.064278,0.261052,0.231646,0.014615,2.401360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Lexington,222167.0,135320.0,86847.0,11382.0,289200.0,70027.0,21360.0,2943.0,478930.0,0.032033,22718.0,31403,4.129836,0.051232,0.245950,0.167884,0.013247,2.155721
96,Pensacola,204344.0,148104.0,56240.0,28687.0,308600.0,78315.0,12679.0,4051.0,455139.0,0.056918,32809.0,39853,3.940497,0.140386,0.225445,0.221527,0.019824,2.227318
97,Visalia,145442.0,86036.0,59406.0,9803.0,339800.0,73097.0,15209.0,1510.0,442179.0,0.022043,23492.0,16052,4.648618,0.067401,0.256018,0.273048,0.010382,3.040243
98,Fayetteville,150658.0,88351.0,62307.0,15242.0,216900.0,61220.0,14154.0,2307.0,440121.0,0.018102,20811.0,23193,3.542960,0.101170,0.227165,0.235549,0.015313,2.921325


### Calculate Composite Index

```python
Housing_Market_Index = (
    0.25 * Recent_Units_Share +
    0.20 * (New_Housing_Units / Total_Housing_Units) +
    0.20 * (MedianHouseholdIncome / Median_Home_Value) +
    0.15 * (1 - High_Rent_Burden) +
    0.10 * (1 - Owner_Cost_Burden) +
    0.10 * (1 - Vacancy_Rate)
)
```

In [231]:
from sklearn.preprocessing import MinMaxScaler

# Define features used in the index
features = {
    'Recent_Units_Share': housing_index_df['Recent_Units_Share'],
    'New_Units_Rate': housing_index_df['New_Housing_Units'] / housing_index_df['Total_Housing_Units'],
    'Affordability': housing_index_df['MedianHouseholdIncome'] / housing_index_df['Median_Home_Value'],
    'Rent_Burden_Inverse': 1 - housing_index_df['Rent_Burden_Rate'],
    'Owner_Burden_Inverse': 1 - housing_index_df['Owner_Burden_Rate'],
    'Vacancy_Inverse': 1 - housing_index_df['Vacancy_Rate']
}

# Create DataFrame of these features
feature_df = pd.DataFrame(features)

# Apply Min-Max Normalization to scale everything between 0 and 1
scaler = MinMaxScaler()
normalized_features = pd.DataFrame(
    scaler.fit_transform(feature_df),
    columns=feature_df.columns,
    index=feature_df.index
)

# Weighted Composite Index Calculation
housing_index_df['Housing_Market_Index'] = (
    0.25 * normalized_features['Recent_Units_Share'] +
    0.20 * normalized_features['New_Units_Rate'] +
    0.20 * normalized_features['Affordability'] +
    0.15 * normalized_features['Rent_Burden_Inverse'] +
    0.10 * normalized_features['Owner_Burden_Inverse'] +
    0.10 * normalized_features['Vacancy_Inverse']
)

housing_index_df

,Metro_Area,Total_Housing_Units,Owner occupied,Renter occupied,Vacant,Median_Home_Value,MedianHouseholdIncome,High_Rent_Burden,New_Housing_Units,Population,Percent Change in Population (2020-2024),Owner_Cost_Burden,Recent_Units_Share,Affordability_Ratio,Vacancy_Rate,Rent_Burden_Rate,Owner_Burden_Rate,New_Units_Rate,Pop_to_Unit_Ratio,Housing_Market_Index
0,New York,7490498.0,3825885.0,3664613.0,593023.0,610200.0,95220.0,983573.0,56661.0,19052774.0,-0.007054,1220190.0,540955,6.408318,0.079170,0.268398,0.318930,0.007564,2.543592,0.414899
1,Los Angeles,4542998.0,2189902.0,2353096.0,285894.0,867200.0,91960.0,686139.0,31151.0,12828837.0,-0.020709,747559.0,318295,9.430187,0.062931,0.291590,0.341366,0.006857,2.823870,0.271067
2,Chicago,3671659.0,2420840.0,1250819.0,240243.0,316500.0,87071.0,285897.0,18511.0,9461105.0,-0.004315,597290.0,246398,3.634965,0.065432,0.228568,0.246728,0.005042,2.576793,0.463718
3,Dallas,2959385.0,1769613.0,1189772.0,212902.0,373900.0,86860.0,295171.0,78705.0,6366542.0,0.092524,436350.0,724661,4.304628,0.071941,0.248090,0.246579,0.026595,2.151306,0.668770
4,Philadelphia,2484152.0,1668735.0,815417.0,159676.0,356700.0,86867.0,212866.0,36307.0,5965343.0,0.013670,386556.0,188296,4.106277,0.064278,0.261052,0.231646,0.014615,2.401360,0.439316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Lexington,222167.0,135320.0,86847.0,11382.0,289200.0,70027.0,21360.0,2943.0,478930.0,0.032033,22718.0,31403,4.129836,0.051232,0.245950,0.167884,0.013247,2.155721,0.428307
96,Pensacola,204344.0,148104.0,56240.0,28687.0,308600.0,78315.0,12679.0,4051.0,455139.0,0.056918,32809.0,39853,3.940497,0.140386,0.225445,0.221527,0.019824,2.227318,0.432119
97,Visalia,145442.0,86036.0,59406.0,9803.0,339800.0,73097.0,15209.0,1510.0,442179.0,0.022043,23492.0,16052,4.648618,0.067401,0.256018,0.273048,0.010382,3.040243,0.323970
98,Fayetteville,150658.0,88351.0,62307.0,15242.0,216900.0,61220.0,14154.0,2307.0,440121.0,0.018102,20811.0,23193,3.542960,0.101170,0.227165,0.235549,0.015313,2.921325,0.430268
